In [455]:
import os
from os.path import join as pathjoin

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

In [451]:
# All the code for rasterizing image:

def crop_image(img, y, x, h, w):
    """
    Crop the image with given top-left anchor and corresponding width & height
    :param img: image to be cropped
    :param y: height of anchor
    :param x: width of anchor
    :param h: height of the patch
    :param w: width of the patch
    :return:
    """
    if len(img.shape) == 2:
        return img[y:y+w, x:x+h]
    else:
        return img[y:y+w, x:x+h, :]


def make_grid(tile_size, patch_size, overlap=0):
    """
    Extract patches at fixed locations. Output coordinates for Y,X as a list (not two lists)
    :param tile_size: size of the tile (input image)
    :param patch_size: size of the output patch
    :param overlap: #overlapping pixels
    :return:
    """
    max_h = int(tile_size[0] - patch_size[0])
    max_w = int(tile_size[1] - patch_size[1])

    if max_h > 0 and max_w > 0:
        h_step = int(np.ceil(tile_size[0] / (patch_size[0] - overlap)))
        w_step = int(np.ceil(tile_size[1] / (patch_size[1] - overlap)))
    else:
        h_step = 1
        w_step = 1
    patch_grid_h = np.floor(np.linspace(0, max_h, h_step)).astype(np.int32)
    patch_grid_w = np.floor(np.linspace(0, max_w, w_step)).astype(np.int32)

    y, x = np.meshgrid(patch_grid_h, patch_grid_w)

    return list(zip(y.flatten(), x.flatten()))


def patch_tile(rgb, gt, patch_size, pad=0, overlap=0):
    """
    Extract the given rgb and gt tiles into patches
    :param rgb:
    :param gt:
    :param patch_size: size of the patches, should be a tuple of (h, w)
    :param pad: #pixels to be padded around each tile, should be either 
    one element or four elements
    :param overlap: #overlapping pixels between two patches in both vertical
    and horizontal direction
    :return: rgb and gt patches as well as coordinates
    """
    # rgb = misc_utils.load_file(rgb_file)
    # gt = misc_utils.load_file(gt_file)[:, :, 0]
    np.testing.assert_array_equal(rgb.shape[:2], gt.shape)
    grid_list = make_grid(
        np.array(rgb.shape[:2]) + 2 * pad, patch_size, overlap)
    
    for y, x in grid_list:
        rgb_patch = crop_image(
            rgb, y, x, patch_size[0], patch_size[1])
        gt_patch = crop_image(
            gt, y, x, patch_size[0], patch_size[1])

        yield rgb_patch, gt_patch, y, x


def read_geotiff(geotiff_path):
    """
    Read geotiff, return reshaped image and metadata.
    """
    with rasterio.open(geotiff_path, 'r') as src:
        img = src.read()
        img_meta = src.meta
    return reshape_as_image(img), img_meta

def read_labels(labels_path, geotiff_crs):
    """
    Read geojson labels and convert projection, return geopandas dataframe.
    """
    labels = gpd.read_file(labels_path)
    print(labels)
    labels = labels[labels.geometry.notnull()]#[labels.building == 'yes']
    lb = labels.to_crs({'init': geotiff_crs['init']})
    return lb

def make_dir_if_not_exists(path, return_path=False):
    if not os.path.exists(path):
        os.makedirs(path)
    if return_path:
        return path

def save_image(img, path, name):
    make_dir_if_not_exists(path)
    data = Image.fromarray(img.astype(np.uint8))
    data.save(pathjoin(path, name))

def rasterize_labels(labels, img_size,img_meta):
    """
    Draw rasterized labeled imagery based on corresponding geotiff image size.
    :param labels: geopandas dataframe, must have 'geometry' column with Polygon objects
    :img_size: corresponding geotiff image size
    """
    new_polygons = []

    for _, row in labels.iterrows():
        if isinstance(row['geometry'], Polygon):
            new_polygons.append(convert_polygon(
                row['geometry'], img_meta['transform']))
        elif isinstance(row['geometry'], MultiPolygon):
            for poly in list(row['geometry']):
                new_polygons.append(convert_polygon(
                    poly, img_meta['transform']))
        else:
            continue
    return rasterize(shapes=new_polygons, out_shape=img_size)

def convert_polygon(rowcol_polygon, transform):
    """
    Convert polygons from geojson rowcol coordinates to pixel positions
    :param rowcol_polygon: geojson polygon(s)
    :param transform: affine.Affine object, read from geotiff meta
    """
    polygon_points = []

    for point in np.array(rowcol_polygon.exterior.coords):
        # transform rowcol coords to geotiff crs, using reverse affine transformation
        polygon_points.append(~transform * point)

    return Polygon(polygon_points)

In [452]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from OSMPythonTools.api import Api

'''This file contains the following functions:

1. get_ids(geojson_file_path)

Given a file_path, gets the ids for all the *ways* it contains
Each id will be an input to the next function:

2. get_coord(way_id)

Returns a list with the coordinates of the vertices of this way.
This function can be mapped on the output of the previous function
to return a list of the coordinates of each *way*.

The output of the get_coord function serves as an input to the next function:

3. bound_box(list_of_coordinates)

Constructs a box which bounds a *way* object and returns it as a Polygon object.

'''

def get_ids(geojson_file_path):
    '''
    Given a geojson file path extracted from OCM
    with different identified structures (*ways*),
    this function returns a list with their ids.
    '''

    #Read geojson file
    geojson = gpd.read_file(geojson_file_path)

    #Get the id colum and transform it in a list
    ids = list(geojson.loc[:,'id'])

    #Return list with ids
    return(ids)


def get_coord(way_id):
    '''
    Given the id of one identified structure (*way*),
    from OCM, this function returns a list with its coordinates
    '''

    #Define empty list to store coordinates
    coord = []

    #Execute query using OCM API
    #Requires command
    #from OSMPythonTools.api import Api
    api = Api()
    query_results = api.query(way_id)
    nodes = query_results.nodes()

    #Get the coordinates for each node and store in list
    for node in nodes:
        coord.append(node.geometry()['coordinates'])

    return(coord)


def bound_box(list_of_coordinates):
    '''Given a list of coordinates
    (such as the output of function get_coords)
    returns a rectangle which contains the polygon
    defined by the coordinates.
    This rectangle is itself returned as a polygon object.
    Note that this is not the minimum bound box,
    but rather the minimum bound box which is parallel to the x and y axis (parallels and meridians).

    Requires
    from shapely.geometry import Polygon.'''

    #Collects x's and y's of polygons vertices
    x = []
    y = []
    for vertex in list_of_coordinates:
        x.append(vertex[0])
        y.append(vertex[1])

    #Define minimum and maximum values of x and y
    min_x = min(x)
    min_y = min(y)
    max_x = max(x)
    max_y = max(y)

    #Define vertices of box
    #Note that these are not point objects
    #as polygon objects in shapely are not created from point objects
    point_A = (min_x, min_y)
    point_B = (min_x, max_y)
    point_C = (max_x, max_y)
    point_D = (max_x, min_y)

    #Define bound box object as Polygon
    box = Polygon([point_A, point_B, point_C, point_D])

    #Return bound_box
    return(box)

In [1175]:
def get_image(filename,img_path,json_filepath):
    patch_size = (2795,2847)
    # This will read the image and the meta data

    img,img_meta = read_geotiff(img_path)
    # get the image sizes
    img_size = (img_meta['height'], img_meta['width'])
    # readthe crs
    #labels = read_labels('jsons/{}.json'.format(filename),  img_meta['crs'])
    labels = read_labels(json_filepath,  img_meta['crs'])
    gt =rasterize_labels(labels, img_size,img_meta)
    labels
    img_patches_dir = 'images/patches/rgb'
    gt_patches_dir = 'images/patches/gt'
    loc='nia'
    idx=0
    for img_patch, gt_patch, y, x in patch_tile(img, gt, patch_size):
        idx+=1
        img_patchname='img-{}-{}.png'.format(filename,idx)
        gt_patchname='gt-{}-{}.png'.format(filename,idx)
        print(gt_patchname)
        save_image(img_patch, img_patches_dir, img_patchname)
        save_image(gt_patch*255, gt_patches_dir, gt_patchname)
    pass

In [473]:
import ee
import os
from ee import batch
ee.Initialize()

In [1179]:
def getSarTask(coords,name):
    
    imgVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .select('VV') 
    
    desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
    asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));
    
    spring = ee.Filter.date('2016-01-01', '2017-1-01');
    descChange = ee.Image.cat(desc.filter(spring).mean());
    ascChange = ee.Image.cat(asc.filter(spring).mean());
    
    geom = ee.Geometry.Rectangle([coords[0]-0.01,coords[1]-0.01,coords[2]+0.01,coords[3]+0.01]);
    
    image = desc.mosaic()
    image = image.clip(geom)
    image = image.visualize(bands=['VV'],min=-25,max=5)
    
    task = ee.batch.Export.image.toCloudStorage(image=image,
                                        region=image.geometry().bounds().getInfo()['coordinates'],
                                        description='power_plant',
                                        outputBucket='earth_engine_data',
                                        fileNamePrefix=name+"_sar",
                                        scale=10)
    return task

In [1180]:
def getLandsatTask(coords,name):
    def maskL8sr(image):
        # Bits 3 and 5 are cloud shadow and cloud, respectively.
        cloudShadowBitMask = (1 << 3)
        cloudsBitMask = (1 << 5)
        # Get the pixel QA band.
        qa = image.select('pixel_qa')
        # Both flags should be set to zero, indicating clear conditions.
        a = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
        b = qa.bitwiseAnd(cloudsBitMask).eq(0)
        mask = (a and b)
        return image.updateMask(mask).divide(10000)
    
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                  .filterDate('2016-01-01', '2016-12-31') #.map(maskL8sr)
    
    bands = ['B4', 'B3', 'B2'] 
    collection = collection.select(bands)
    
    #geom = ee.Geometry.Rectangle([coords[0],coords[1],coords[2],coords[3]])
    geom = ee.Geometry.Rectangle([coords[0]-0.01,coords[1]-0.01,coords[2]+0.01,coords[3]+0.01]);
    
    image = collection.sort('system:index', False).mosaic()
    image = image.clip(geom)
    image = image.select('B.+')
    
    imageRGB = image.visualize(bands=bands,min=0,max=3000,gamma=1.4)
    
    task = ee.batch.Export.image.toCloudStorage(scale=10,
                                                image=imageRGB,                                        
                                                description='power_plant',
                                                fileFormat = 'GeoTIFF',
                                                outputBucket='earth_engine_data',
                                                fileNamePrefix=name+"_landsat",
                                                region=image.geometry().bounds().getInfo()['coordinates']
                                                )
    return task

In [1181]:
def getNaipTask(coords,name):
    # takes as input coordinates of the boundary to export to 
    # google cloud bucket
    geom = ee.Geometry.Rectangle([coords[0],coords[1],coords[2],coords[3]]);
    geom = ee.Geometry.Rectangle([coords[0]-0.01,coords[1]-0.01,coords[2]+0.01,coords[3]+0.01]);
    collection = ee.ImageCollection("USDA/NAIP/DOQQ") \
                .filter(ee.Filter.date('2015-01-01', '2017-12-31'));
    trueColor = collection.select(['R', 'G', 'B','N'])
    trueColorVis = {
      min: 0.0,
      max: 255.0,
    }
    image = collection.sort('system:index', False).mosaic()
    image = image.clip(geom)
    image.projection()
    
    task = ee.batch.Export.image.toCloudStorage(image=image,
                                                region=image.geometry().bounds().\
                                                getInfo()['coordinates'],
                                                description='power_plant',
                                                outputBucket='earth_engine_data',
                                                fileNamePrefix=name+"_naip",
                                                scale=10)
    return task

def downloadGStorage(buck_name, local_addr):
    os.system('export GOOGLE_APPLICATION_CREDENTIALS=\"key.json\"')
    os.system('gsutil cp -r '+ buck_name + ' ' + local_addr)
    pass

Main demonstration:

https://overpass-turbo.eu/s/QBq
 
The below geojson was taken from the link above:

simply open the link and open and exprot the geojson


For a solar panel ground truth set:
http://overpass-turbo.eu/s/R6y


In [1168]:
plant_name = "harris_nuclear_plant"
example_file = "jsons/"+plant_name+".geojson"

#This geojson file contains the following *ways* in it:
ids = get_ids(example_file)
#print(ids)

#The coordinates of the first *way* in the geojson file above is:
coords = get_coord(ids[0])
coords[0] = [coords[0][0]-0.01,coords[0][1]-0.01]
coords[0] = [coords[1][0]-0.01,coords[1][1]-0.01]

#We can also get the coordinates of all the *ways* in the previous file:
all_coords = list(map(get_coord, ids))

# These are the coordinates of the bounding box
np_bbox = np.array(bound_box(coords).exterior)

a = list(np_bbox[0,:])
b = list(np_bbox[2,:])
a.extend(b)
print(a)

[-78.96836900000001, 35.621662, -78.953271, 35.635666]


In [1182]:
print(a)
print(plant_name)
task1 = getSarTask(a,plant_name)
task2 = getLandsatTask(a,plant_name)
task3 = getNaipTask(a,plant_name)

[-78.96836900000001, 35.621662, -78.953271, 35.635666]
harris_nuclear_plant


In [1183]:
task1.start()
task2.start()

In [1184]:
# Step 1: download image from earth engine to google cloud
task3.start()

In [1191]:
task3.status()

{'creation_timestamp_ms': 1586976392112,
 'description': 'power_plant',
 'id': 'QJMB4RVU2KC733MZRKRRPDEZ',
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1586976392112}

In [1161]:
# Step 2: download image from google cloud bucket to local directory
downloadGStorage('gs://earth_engine_data', 
                 'gee_images/')

In [1176]:
# this will fetch the image hard coded from images/harris_nuclear.tif
# and rasterize the image
get_image(plant_name,'gee_images/earth_engine_data/'+plant_name+'_naip.tif',example_file)

             id           @id generator:method                  name  \
0  way/32248471  way/32248471          fission  Harris Nuclear Plant   

      operator plant:output:electricity plant:source  power  wikidata  \
0  Duke Energy                   900 MW      nuclear  plant  Q1568903   

                               wikipedia  \
0  en:Shearon Harris Nuclear Power Plant   

                                            geometry  
0  POLYGON ((-78.95745 35.63010, -78.95423 35.631...  
gt-harris_nuclear_plant-1.png


/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [1172]:
example_file

'jsons/harris_nuclear_plant.geojson'